### Read LABS from 'labs_paulis'

In [14]:
import json
import numpy as np
# Read from JSON file
num_qubit = 10  # Length of each vector
with open("LABS/labs_paulis/"+ str(num_qubit) +"_"+ str(1) +".json", "r") as file:
    boolean_functions = json.load(file)

F = []
one_term_boolean_functions = []
for f in boolean_functions:
    if sum(f)==0:
        print("Incorrect! Containing all False boolean function")
        break
    if sum(f)==1:
        one_term_boolean_functions.append(f)
    if sum(f)>1:
        F.append(f)

In [15]:
import numpy as np

def hamming_distance(vec1, vec2):
    """
    Computes the Hamming distance between two binary vectors.
    
    :param vec1: First binary vector (list of 0/1 or True/False)
    :param vec2: Second binary vector (list of 0/1 or True/False)
    :return: Integer Hamming distance
    """
    return sum(a != b for a, b in zip(vec1, vec2))

def select_best_vector(reference_list, candidate_list):
    """
    Selects the vector from candidate_list that has the minimum total Hamming weight 
    with respect to all vectors in reference_list.
    
    :param reference_list: List of binary vectors (comparison base)
    :param candidate_list: List of binary vectors (selection pool)
    :return: The best binary vector from candidate_list
    """
    best_vector = None
    min_hamming_sum = float('inf')

    for candidate in candidate_list:
        total_hamming = sum(hamming_distance(candidate, ref) for ref in reference_list)
        
        if total_hamming < min_hamming_sum:
            min_hamming_sum = total_hamming
            best_vector = candidate

    return best_vector

ordered_F = []
candidate_F = [f for f in F]
while candidate_F:
    # Select the best vector
    best_vector = select_best_vector(ordered_F, candidate_F)
    ordered_F.append(best_vector)
    candidate_F.remove(best_vector)

In [16]:
from QAOA_benchmark import boolean_func_to_normal_qaoa_circuit
import time
start_time = time.time()
org_qc, theta_2 = boolean_func_to_normal_qaoa_circuit(ordered_F,1)

parameter_values = [np.pi/5, np.pi/4]
parameter_dict = {param: value for param, value in zip(theta_2, parameter_values)}

# Bind the parameters
bound_org_qc = org_qc.assign_parameters(parameter_dict)

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=3)
candidate_circuit = pm.run(bound_org_qc)

end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)

0.028293132781982422


In [17]:
from qiskit import QuantumCircuit, qpy
with open("LABS"+"/logical/"+ str(candidate_circuit.num_qubits) +"_logical_circuit.qpy", "wb") as qpy_file_write:
    qpy.dump(candidate_circuit, qpy_file_write)